In [2]:
import pandas as pd
from scipy import sparse
import pickle

In [3]:
ratings = pd.read_csv('dataset/ratings.csv')
movies = pd.read_csv('dataset/movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()


(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [4]:
Ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
Ratings.head()
print("Before: ",Ratings.shape)
userRatings = Ratings.dropna(thresh=10, axis=1).fillna(0,axis=1)
print("After: ",Ratings.shape)

Before:  (610, 9719)
After:  (610, 9719)


In [5]:
Matrix = Ratings.corr(method='pearson')
Matrix.head(100)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3 Ninjas (1992),NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-1.0,-0.904534,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,-1.0,NaN
3 Ninjas Kick Back (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3 Ninjas Knuckle Up (1995),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def similar(movie_name,rating):
    similar_sc = Matrix[movie_name]*(rating-2.5)
    similar_sc = similar_sc.sort_values(ascending=False)
    return similar_sc

In [7]:
romantic= [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similarmovie = pd.DataFrame()
for movie,rating in romantic:
    similarmovie= similarmovie.append(similar(movie,rating),ignore_index = True)

similarmovie.head(10)

,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
0,NaN,NaN,NaN,NaN,NaN,NaN,2.492941,NaN,2.500000,2.362278,...,NaN,-2.165064,NaN,NaN,NaN,-0.972207,2.079645,2.50,1.430194,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367095,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.500000,0.125805,NaN,0.500000,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,-0.365765,NaN,-0.527162,-0.367016,...,NaN,-0.149010,NaN,NaN,NaN,0.115370,-0.510536,0.75,0.468534,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,-0.011726,NaN,-0.099408,-0.115385,...,NaN,-0.288278,-0.5,NaN,NaN,0.153106,-0.021188,0.50,0.070999,NaN


In [8]:
similarmovie.sum().sort_values(ascending=False).head(20)

Raiders of the Lost Ark: The Adaptation (1989)             5.000000
Magic Mike (2012)                                          5.000000
Fool's Gold (2008)                                         5.000000
Memories (Memorîzu) (1995)                                 4.989676
Wanderlust (2012)                                          4.814881
For Richer or Poorer (1997)                                4.750833
Sunshine Cleaning (2008)                                   4.575480
Ferngully: The Last Rainforest (1992)                      4.566931
District 13: Ultimatum (Banlieue 13 - Ultimatum) (2009)    4.500000
Exam (2009)                                                4.500000
Bad Words (2013)                                           4.500000
Cedar Rapids (2011)                                        4.500000
Wicker Park (2004)                                         4.500000
Max Keeble's Big Move (2001)                               4.500000
Helvetica (2007)                                

In [9]:
action= [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similarmovie= pd.DataFrame()
for movie,rating in action:
    similarmovie= similarmovie.append(similar(movie,rating),ignore_index =True)

similarmovie.head(10)
similarmovie.sum().sort_values(ascending=False).head(20)

Solo: A Star Wars Story (2018)                  5.927407
Avengers, The (1998)                            5.750000
Monkeybone (2001)                               5.522594
Taken 3 (2015)                                  5.502159
Garfield: The Movie (2004)                      5.500000
Great Mouse Detective, The (1986)               5.500000
Black Dahlia, The (2006)                        5.500000
Halloween: Resurrection (Halloween 8) (2002)    5.500000
Spice World (1997)                              5.442692
Heavy Metal (1981)                              5.391611
Mortal Kombat: Annihilation (1997)              5.183170
Ghosts of Mars (2001)                           5.128734
Joy Ride (2001)                                 5.095006
Switch, The (2010)                              5.089785
Searching for Bobby Fischer (1993)              5.087619
Batman: Year One (2011)                         5.061383
Audition (Ôdishon) (1999)                       5.048586
Hollywood Homicide (2003)      

In [10]:
import pickle

In [11]:
pickle.dump(Matrix,open('cmll.pkl','wb'))

In [12]:
alldict=movies['title'].to_dict()

In [13]:
alldict

{0: 'Toy Story (1995)',
 1: 'Jumanji (1995)',
 2: 'Grumpier Old Men (1995)',
 3: 'Waiting to Exhale (1995)',
 4: 'Father of the Bride Part II (1995)',
 5: 'Heat (1995)',
 6: 'Sabrina (1995)',
 7: 'Tom and Huck (1995)',
 8: 'Sudden Death (1995)',
 9: 'GoldenEye (1995)',
 10: 'American President, The (1995)',
 11: 'Dracula: Dead and Loving It (1995)',
 12: 'Balto (1995)',
 13: 'Nixon (1995)',
 14: 'Cutthroat Island (1995)',
 15: 'Casino (1995)',
 16: 'Sense and Sensibility (1995)',
 17: 'Four Rooms (1995)',
 18: 'Ace Ventura: When Nature Calls (1995)',
 19: 'Money Train (1995)',
 20: 'Get Shorty (1995)',
 21: 'Copycat (1995)',
 22: 'Assassins (1995)',
 23: 'Powder (1995)',
 24: 'Leaving Las Vegas (1995)',
 25: 'Othello (1995)',
 26: 'Now and Then (1995)',
 27: 'Persuasion (1995)',
 28: 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 29: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 30: 'Dangerous Minds (1995)',
 31: 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995

In [14]:
pickle.dump(alldict,open('allmovie_dict.pkl','wb'))

In [16]:
pickle.dump(similarmovie,open('movies.pkl','wb'))

In [17]:
newlist=list(similarmovie.columns.values)

In [18]:
newdict={ stu : "Passed" for stu in newlist }

In [19]:
newdict

{"'71 (2014)": 'Passed',
 "'Hellboy': The Seeds of Creation (2004)": 'Passed',
 "'Round Midnight (1986)": 'Passed',
 "'Salem's Lot (2004)": 'Passed',
 "'Til There Was You (1997)": 'Passed',
 "'Tis the Season for Love (2015)": 'Passed',
 "'burbs, The (1989)": 'Passed',
 "'night Mother (1986)": 'Passed',
 '(500) Days of Summer (2009)': 'Passed',
 '*batteries not included (1987)': 'Passed',
 '...All the Marbles (1981)': 'Passed',
 '...And Justice for All (1979)': 'Passed',
 '00 Schneider - Jagd auf Nihil Baxter (1994)': 'Passed',
 '1-900 (06) (1994)': 'Passed',
 '10 (1979)': 'Passed',
 '10 Cent Pistol (2015)': 'Passed',
 '10 Cloverfield Lane (2016)': 'Passed',
 '10 Items or Less (2006)': 'Passed',
 '10 Things I Hate About You (1999)': 'Passed',
 '10 Years (2011)': 'Passed',
 '10,000 BC (2008)': 'Passed',
 '100 Girls (2000)': 'Passed',
 '100 Streets (2016)': 'Passed',
 '101 Dalmatians (1996)': 'Passed',
 '101 Dalmatians (One Hundred and One Dalmatians) (1961)': 'Passed',
 "101 Dalmatians I

In [20]:
pickle.dump(newdict,open('movie_dict.pkl','wb'))

In [21]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
